# dask
> dask tests

Exploring distributed computation using dask platform.

Dask (https://www.dask.org) seems to be an easy approach how to scale up the calculation that are done using 
pytorch and numpy. 

Dask has a similar interface as numpy and torch. Dask breaks up large data frames into sections and calculations 
are automatically distributed across many procesess.

In [ ]:
%pip install 'dask[complete]'


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Clinet

This is the root of our distributed computation

In [ ]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

/Users/jstranik/Developer/manafund/mana-signals/.venv/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50914 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.1.94:50914/status,
Dashboard: http://192.168.1.94:50914/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://192.168.1.94/93407/10,Workers: 1
Dashboard: http://192.168.1.94:50914/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: inproc://192.168.1.94/93407/13,Total threads: 4
Dashboard: http://192.168.1.94:50915/status,Memory: 1.86 GiB
Nanny: None,


2024-03-05 16:16:32,954 - distributed.scheduler - ERROR - Couldn't gather keys: {('sum-aggregate-2151311f9cb12b4138e475aa03fb945e',): 'waiting'}
2024-03-05 16:16:33,348 - distributed.scheduler - ERROR - Couldn't gather keys: {('sum-aggregate-2151311f9cb12b4138e475aa03fb945e',): 'waiting'}
2024-03-05 16:16:34,255 - distributed.scheduler - ERROR - Couldn't gather keys: {('sum-aggregate-2151311f9cb12b4138e475aa03fb945e',): 'waiting'}
2024-03-06 07:39:33,252 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'inproc://192.168.1.94/93407/13', name: 0, status: running, memory: 100, processing: 0>
2024-03-06 07:39:33,356 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'inproc://192.168.1.94/93407/13'.
2024-03-06 07:39:33,358 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.


---
Our fist testing distributed calculation

In [ ]:
import dask.array as da
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [ ]:
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

dask.array<mean_agg-aggregate, shape=(5000,), dtype=float64, chunksize=(500,), chunktype=numpy.ndarray>

In [ ]:
z.compute()

array([0.99981844, 1.00048762, 1.00448889, ..., 0.99933693, 0.99871619,
       1.00062619])

In [ ]:
%timeit y.sum().compute()

2024-03-05 16:16:32,954 - distributed.client - WARNING - Couldn't gather 1 keys, rescheduling [('sum-aggregate-2151311f9cb12b4138e475aa03fb945e',)]
2024-03-05 16:16:33,348 - distributed.client - WARNING - Couldn't gather 1 keys, rescheduling [('sum-aggregate-2151311f9cb12b4138e475aa03fb945e',)]


The slowest run took 103.15 times longer than the fastest. This could mean that an intermediate result is being cached.
111 ms ± 165 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
y2 = y.persist()

In [ ]:
%timeit y2.sum().compute()

2024-03-05 16:16:34,257 - distributed.client - WARNING - Couldn't gather 1 keys, rescheduling [('sum-aggregate-2151311f9cb12b4138e475aa03fb945e',)]


The slowest run took 26.18 times longer than the fastest. This could mean that an intermediate result is being cached.
47 ms ± 35.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
import dask
dask.datasets.make_people()

RuntimeError: The mimesis module is required for this function.  Try:
  python -m pip install mimesis

In [ ]:
%pip install mimesis

  Obtaining dependency information for mimesis from https://files.pythonhosted.org/packages/7e/84/dda8613360ef90a832d3e984ce566faea75d2b98a8c02d0bf7e8e343eb90/mimesis-15.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 8.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import dask
mp = dask.datasets.make_people(); 


In [ ]:
len(mp.compute()

10000